

# Basic AequilibraE use

In this example, we grab a pre-existing model and do the following:

* Add new fields to the network
* Export existing matrices to OMX
* Perform Skimming
* Perform Assignment


### Before we begin
Let's make a copy of the model so we can make the changes we want without overwriting the original model

In [1]:
import shutil

base_pth = r"G:\My Drive\DATA\Pedro\Research\CONFERENCES\TRB ITM\2023\LongAn_base_model"
model_path = r"C:\Users\Pedro\AppData\Local\Temp\_model_experiments"

shutil.rmtree(model_path,ignore_errors=True)
shutil.copytree(base_pth, model_path)

'C:\\Users\\Pedro\\AppData\\Local\\Temp\\_model_experiments'

## Let's do some modeling

In [2]:
# Imports
from os.path import join
from aequilibrae import Project

In [3]:
# from aequilibrae.matrix import AequilibraeMatrix

# mat = AequilibraeMatrix()
# mat.load(join(base_pth, "matrices", "cvts_trips.aem"))
# mat.names
# mat2 = AequilibraeMatrix()
# mat2.create_empty(file_name=join(base_pth, "matrices", "base_demand.aem"),
#                   zones=mat.zones,
#                   matrix_names=["red_cars_AM", "blue_cars_AM", "Trucks_AM", "red_cars_MD", "blue_cars_MD", "Trucks_MD", "red_cars_PM", "blue_cars_PM", "Trucks_PM"],
#                   index_names=["zones"])

# mat2.matrices[:,:,0] = mat.vehicle_type_1
# mat2.matrices[:,:,1] = mat.vehicle_type_6
# mat2.matrices[:,:,2] = mat.vehicle_type_10 / 2.5
# mat2.matrices[:,:,3] = mat.vehicle_type_2
# mat2.matrices[:,:,4] = mat.vehicle_type_3
# mat2.matrices[:,:,5] = mat.vehicle_type_4 / 5
# mat2.matrices[:,:,6] = mat.vehicle_type_5
# mat2.matrices[:,:,7] = mat.vehicle_type_7
# mat2.matrices[:,:,8] = mat.vehicle_type_8 / 2
# mat2.index[:] = mat.index[:]
# mat2.export(join(base_pth, "matrices", "base_demand.aem"))
# print(mat.matrices.sum())
# mat.close()
# mat2.close()

In [4]:
# We open a project
project = Project.from_path(model_path)

## Model stats

In [5]:
print(f"Links: {project.network.count_links():,}")
print(f"Nodes: {project.network.count_nodes():,}")
print(f"Zones: {project.network.count_centroids():,}")

Links: 384,122
Nodes: 369,872
Zones: 698


Let's see which fields we have in our links layer

In [6]:
project.network.links.fields.all_fields()

['a_node',
 'b_node',
 'direction',
 'distance',
 'link_id',
 'link_type',
 'modes',
 'name',
 'speed',
 'capacity',
 'lanes',
 'geometry',
 'osm_id']

We are missing travel time. Let's add it

In [7]:
fields = project.network.links.fields
fields.add("ff_ttime_ab", description="AB Free flow travel time")
fields.add("ff_ttime_ba", description="BA Free flow travel time")
fields.save()

In [8]:
project.network.links.fields.all_fields()

['a_node',
 'b_node',
 'capacity',
 'direction',
 'distance',
 'geometry',
 'lanes',
 'link_id',
 'link_type',
 'modes',
 'name',
 'osm_id',
 'speed',
 'ff_ttime']

#### Let's use some SQL to compute these fields

Speeds are in KM/h and distances are ALWAYS in meters in AequilibraE


In [9]:
%%time
sql = """Update links set ff_ttime_ab=(distance/1000)/speed_ab * 60, 
                          ff_ttime_ba=(distance/1000)/speed_ab * 60"""
project.conn.execute(sql)
project.conn.commit()

CPU times: total: 703 ms
Wall time: 2.03 s


## Computes Graph



In [10]:
%%time
project.network.build_graphs(modes=["c"])

CPU times: total: 1.39 s
Wall time: 3.97 s


In [11]:
graph = project.network.graphs["c"]
graph.set_graph("ff_ttime")
graph.set_skimming(["distance", "ff_ttime"])

## Skimming

In [12]:
from aequilibrae.paths import NetworkSkimming

In [13]:
%%time
skimmer = NetworkSkimming(graph)
skimmer.execute()

CPU times: total: 8.31 s
Wall time: 805 ms


In [14]:
skimmer.results.skims.distance

array([[0.00000000e+00, 2.08210301e+05, 1.96844618e+06, ...,
        2.30752212e+05, 2.31110720e+05, 2.26458067e+05],
       [2.47165989e+05, 0.00000000e+00, 2.04366321e+06, ...,
        3.05969242e+05, 3.06327750e+05, 3.01675097e+05],
       [1.96299604e+06, 2.03601550e+06, 0.00000000e+00, ...,
        1.75525994e+06, 1.75561845e+06, 1.75709952e+06],
       ...,
       [2.29378467e+05, 3.02397928e+05, 1.75985749e+06, ...,
        0.00000000e+00, 1.54895959e+03, 3.03003335e+03],
       [2.29626108e+05, 3.02645569e+05, 1.76010513e+06, ...,
        1.43809163e+03, 0.00000000e+00, 1.86561573e+03],
       [2.25019026e+05, 2.98038487e+05, 1.76174415e+06, ...,
        3.07710875e+03, 1.89506487e+03, 0.00000000e+00]])

In [15]:
skimmer.results.skims.ff_ttime

array([[0.00000000e+00, 3.54299335e+02, 2.48121136e+03, ...,
        2.86413631e+02, 2.86151490e+02, 2.88266825e+02],
       [3.53429163e+02, 0.00000000e+00, 2.58233037e+03, ...,
        3.87532636e+02, 3.87270495e+02, 3.89385830e+02],
       [2.50353592e+03, 2.60859558e+03, 0.00000000e+00, ...,
        2.25222599e+03, 2.25196385e+03, 2.25514854e+03],
       ...,
       [2.78741510e+02, 3.83801176e+02, 2.22291488e+03, ...,
        0.00000000e+00, 2.64542158e+00, 5.83011813e+00],
       [2.78311411e+02, 3.83371077e+02, 2.22248479e+03, ...,
        2.47746350e+00, 0.00000000e+00, 3.70308397e+00],
       [2.81720163e+02, 3.86779829e+02, 2.22591373e+03, ...,
        5.90640663e+00, 3.74153842e+00, 0.00000000e+00]])

We can save this matrix to disk and add to the model

In [16]:
skimmer.save_to_project("skim_matrix")

## Matrices

In [17]:
project.matrices.list()

,name,file_name,cores,procedure,procedure_id,timestamp,description,status
0,base_demand,base_demand.aem,9,None,None,2023-05-30 03:52:33,None,
1,skim_matrix,skim_matrix.omx,2,Network skimming,3e6a598e887b4195863ae15e2432d455,2023-05-30 14:23:27.520067,None,


In [18]:
mat = project.matrices.get_matrix("base_demand")
mat.names

['red_cars_AM',
 'blue_cars_AM',
 'Trucks_AM',
 'red_cars_MD',
 'blue_cars_MD',
 'Trucks_MD',
 'red_cars_PM',
 'blue_cars_PM',
 'Trucks_PM']

Let's export only the matrices for the peak period

In [19]:
mat.export(join(project.matrices.fldr, "peak_demand.omx"), cores=['red_cars_AM', 'blue_cars_AM', 'Trucks_AM', 'red_cars_PM', 'blue_cars_PM', 'Trucks_PM'])

In [20]:
project.matrices.update_database()
project.matrices.list()

,name,file_name,cores,procedure,procedure_id,timestamp,description,status
0,base_demand,base_demand.aem,9,None,None,2023-05-30 03:52:33,None,
1,skim_matrix,skim_matrix.omx,2,Network skimming,3e6a598e887b4195863ae15e2432d455,2023-05-30 14:23:27.520067,None,
2,peak_demand_omx,peak_demand.omx,6,None,None,2023-05-30 04:23:28,None,


## Assignment

Let's assign only a few of the matrices we exported

In [21]:
demand_cars = project.matrices.get_matrix("peak_demand_omx")
demand_cars.computational_view(['red_cars_AM', 'blue_cars_AM'])

In [22]:
demand_trucks = project.matrices.get_matrix("peak_demand_omx")
demand_trucks.computational_view(['Trucks_AM'])

In [23]:
from aequilibrae.paths import TrafficAssignment, TrafficClass

In [24]:

# Create the assignment class
car_class = TrafficClass(name="car", graph=graph, matrix=demand_cars)
truck_class = TrafficClass(name="truck", graph=graph, matrix=demand_trucks)

truck_class.set_pce(1.8)
# truck_class.set_select_links()
# truck_class.set_vot()
# truck_class.set_fixed_cost()

In [25]:

assig = TrafficAssignment()

# The first thing to do is to add at list of traffic classes to be assigned
assig.add_class(car_class)
assig.add_class(truck_class)

# We set these parameters only after adding one class to the assignment
assig.set_vdf("BPR")  # This is not case-sensitive

# Then we set the volume delay function
assig.set_vdf_parameters({"alpha": 0.15, "beta": 4.0})  # And its parameters

assig.set_capacity_field("capacity")  # The capacity and free flow travel times as they exist in the graph
assig.set_time_field("ff_ttime")

# And the algorithm we want to use to assign
assig.set_algorithm("bfw")

# Since I haven't checked the parameters file, let's make sure convergence criteria is good
assig.max_iter = 5
assig.rgap_target = 0.0001

assig.execute()  # we then execute the assignment

In [26]:
assig.report()

,iteration,rgap,alpha,warnings,beta0,beta1,beta2
0,1,inf,1.000000,,1.00000,0.00000,0.0
1,2,0.551868,0.420385,,1.00000,0.00000,0.0
2,3,0.150826,0.184132,,1.00000,0.00000,0.0
3,4,0.037613,0.073601,,1.00000,0.00000,0.0
4,5,0.030225,0.085541,,0.79083,0.20917,0.0


In [27]:
assig.results().max()

red_cars_AM_ab        3402.933333
red_cars_AM_ba        3413.275000
red_cars_AM_tot       6816.208333
blue_cars_AM_ab        880.941667
blue_cars_AM_ba        888.533333
blue_cars_AM_tot      1769.475000
Trucks_AM_ab             9.780088
Trucks_AM_ba            12.630000
Trucks_AM_tot           20.243333
Congested_Time_AB       65.107927
Congested_Time_BA       65.107927
Congested_Time_Max      65.107927
Delay_factor_AB         53.783730
Delay_factor_BA         53.783730
Delay_factor_Max        53.783730
VOC_AB                   4.331140
VOC_BA                   4.331140
VOC_max                  4.331140
PCE_AB                4297.579000
PCE_BA                4324.542333
PCE_tot               8622.121333
dtype: float64

In [28]:
assig.save_results("tutorial")

If you want to show the path in Python
We do NOT recommend this, though....  It is very slow for real networks



In [ ]:
import matplotlib.pyplot as plt
from shapely.ops import linemerge

In [ ]:
# links = project.network.links

# # We plot the entire network
# curr = project.conn.cursor()
# curr.execute("Select link_id from links;")

# for lid in curr.fetchall():
#     geo = links.get(lid[0]).geometry
#     plt.plot(*geo.xy, color="red")

# path_geometry = linemerge(links.get(lid).geometry for lid in res.path)
# plt.plot(*path_geometry.xy, color="blue", linestyle="dashed", linewidth=2)
# plt.show()